# Scraper

In [57]:
import pandas as pd 
import requests
from bs4 import BeautifulSoup

def get_links(): 
    response = requests.get('https://en.wikipedia.org/wiki/List_of_language_names')
    soup = BeautifulSoup(response.content, 'html.parser')
    links = soup.find_all('a')
    links = [link.get('href') for link in links if (link.get('title') is not None) and ('language' in link.get('title'))]
    links = [link for link in links if link.endswith('_language')]
    links = ["https://en.wikipedia.org"+link for link in links]
    return links

urls = get_links()

In [63]:
from tqdm import tqdm

In [65]:
data = []
errors = []

for url in tqdm(urls): 
    try: 
        lang = url.split('/')[-1]

        response = requests.get(url)
        content  = response.content
        parser   = BeautifulSoup(content, 'html.parser')
        table    = parser.find('table', {'class': 'infobox'})
        regions  = table.find_all('td', {'class': 'infobox-data'})[1].text
        speakers = table.find_all('td', {'class': 'infobox-data'})[3].text

        data.append((lang, regions, speakers))
    except Exception as e: 
        errors.append((e, url))
        continue

df = pd.DataFrame(data, columns=['lang', 'regions', 'speakers'])

 21%|██        | 152/720 [00:54<04:00,  2.36it/s]

In [28]:
df

,lang,regions,speakers
0,Urdu,Pakistan (widely used as lingua franca)[a]Indi...,L1: 70 million (2011–2018)[1]L2: 160 million (...
1,Yoruba_language,Yorubaland,L1: 44 million (2020)[1]L2: 2.0 million (no da...


'Pakistan (widely used as lingua franca)[a]India (as a minority in the Hindustani Belt & Deccan)[b]Afghanistan (as a minority across the country)[c]Nepal (as a minority in the Terai)[d]Bangladesh (as a minority in Old Dhaka)[e]'

In [56]:
links

['/wiki/Aari_language',
 '/wiki/Abaza_language',
 '/wiki/Official_language',
 '/wiki/Abellen_language',
 '/wiki/Abenaki_language',
 '/wiki/Abkhaz_language',
 '/wiki/Official_language',
 '/wiki/Abui_language',
 '/wiki/Abure_language',
 '/wiki/Acehnese_language',
 '/wiki/Achang_language',
 '/wiki/Achi_language',
 '/wiki/Minority_language',
 '/wiki/Achumawi_language',
 '/wiki/Adai_language',
 '/wiki/Official_language',
 '/wiki/Adele_language',
 '/wiki/Adjukru_language',
 '/wiki/Adnyamathanha_language',
 '/wiki/Adyghe_language',
 '/wiki/Official_language',
 '/wiki/Afar_language',
 '/wiki/Official_language',
 '/wiki/Minority_language',
 '/wiki/Aguacateco_language',
 '/wiki/Aghul_language',
 '/wiki/Official_language',
 '/wiki/Ahtna_language',
 '/wiki/Official_language',
 '/wiki/Ainu_language',
 '/wiki/Minority_language',
 '/wiki/Official_language',
 '/wiki/Akan_language',
 '/wiki/Official_language',
 '/wiki/Akkadian_language',
 '/wiki/Aklanon_language',
 '/wiki/Official_language',
 '/wiki/Ak